In [13]:
import tkinter as tk
from tkinter import messagebox
import numpy as np
import random

class TicTacToe:
    def __init__(self, root):
        self.root = root
        self.root.title("Tic Tac Toe")
        self.board = np.full((3, 3), '_', dtype=str)
        self.difficulty = tk.StringVar(value="Noob")

        self.create_widgets()
        self.update_board_display()

    def create_widgets(self):
        self.status_label = tk.Label(self.root, text="Your Turn (X)", font=("Arial", 14))
        self.status_label.grid(row=0, column=0, columnspan=3, pady=10)

        self.buttons = [[tk.Button(self.root, text="", font=("Arial", 20), width=5, height=2,
                                   command=lambda r=r, c=c: self.player_move(r, c))
                         for c in range(3)] for r in range(3)]

        for r in range(3):
            for c in range(3):
                self.buttons[r][c].grid(row=r+1, column=c, padx=5, pady=5)

        # Difficulty Options
        tk.Label(self.root, text="Difficulty:", font=("Arial", 12)).grid(row=4, column=0, pady=10)
        tk.OptionMenu(self.root, self.difficulty, "Noob", "Beginner", "Hard").grid(row=4, column=1)

        # Restart & Surrender
        tk.Button(self.root, text="🔁 Restart", command=self.restart).grid(row=4, column=2)
        tk.Button(self.root, text="🏳 Surrender", command=self.surrender).grid(row=5, column=0, columnspan=3, pady=10)

    def update_board_display(self):
        for r in range(3):
            for c in range(3):
                symbol = self.board[r, c]
                self.buttons[r][c]['text'] = '' if symbol == '_' else symbol
                self.buttons[r][c]['state'] = tk.NORMAL if symbol == '_' else tk.DISABLED

    def player_move(self, row, col):
        if self.board[row, col] == '_':
            self.board[row, col] = 'X'
            self.update_board_display()
            if self.check_winner('X'):
                self.end_game("🎉 You Win!")
                return
            elif self.is_draw():
                self.end_game("It's a draw!")
                return
            self.status_label.config(text="Computer's Turn (O)")
            self.root.after(500, self.computer_move)

    def computer_move(self):
        difficulty = self.difficulty.get()
        empty = list(zip(*np.where(self.board == '_')))

        if difficulty == "Noob":
            move = random.choice(empty)
        elif difficulty == "Beginner":
            move = self.block_or_random('X', empty)
        elif difficulty == "Hard":
            move = self.try_win_or_block(empty)

        self.board[move] = 'O'
        self.update_board_display()

        if self.check_winner('O'):
            self.end_game("💻 Computer Wins!")
        elif self.is_draw():
            self.end_game("It's a draw!")
        else:
            self.status_label.config(text="Your Turn (X)")

    def block_or_random(self, target, empty):
        for (r, c) in empty:
            self.board[r, c] = target
            if self.check_winner(target):
                self.board[r, c] = '_'
                return (r, c)
            self.board[r, c] = '_'
        return random.choice(empty)

    def try_win_or_block(self, empty):
        # Try to win
        for (r, c) in empty:
            self.board[r, c] = 'O'
            if self.check_winner('O'):
                self.board[r, c] = '_'
                return (r, c)
            self.board[r, c] = '_'
        # Try to block
        return self.block_or_random('X', empty)

    def check_winner(self, player):
        for i in range(3):
            if all(self.board[i, :] == player) or all(self.board[:, i] == player):
                return True
        if all([self.board[i, i] == player for i in range(3)]) or all([self.board[i, 2 - i] == player for i in range(3)]):
            return True
        return False

    def is_draw(self):
        return not np.any(self.board == '_')

    def end_game(self, msg):
        self.status_label.config(text=msg)
        for row in self.buttons:
            for btn in row:
                btn['state'] = tk.DISABLED
        messagebox.showinfo("Game Over", msg)

    def restart(self):
        self.board = np.full((3, 3), '_', dtype=str)
        self.update_board_display()
        self.status_label.config(text="Your Turn (X)")

    def surrender(self):
        self.end_game("🏳 You Surrendered!")

# Run the game
if __name__ == "__main__":
    root = tk.Tk()
    game = TicTacToe(root)
    root.mainloop()
